### Load and preprocess data

In [111]:
import scipy.io
import numpy as np
import pandas as pd

mat = scipy.io.loadmat('BRCA1View20000.mat')
mat = {k:v for k, v in mat.items() if k[0] != '_'}
print(len(mat['targets']))
#mat = {k: pd.Series(v[0]) for k, v in mat.items() if len(v) == 0}
mat = {k: pd.Series(v.flatten()) for k, v in mat.items()}

targets = pd.DataFrame.from_dict(mat).dropna()[['id', 'targets']]

2133


In [166]:
len(np.unique(targets['targets']))

6

In [113]:
import pandas as pd
import io
import requests
url="https://metabric.s3-us-west-1.amazonaws.com/cat2vec/cat2vec_adj.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [114]:
c['id'] = c['Patient ID'].apply(lambda x: float(x.split("-")[1]))

In [116]:
data = c.merge(labels, on='id')

# Baseline Neural Network Implementation

In [119]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from numpy import genfromtxt
import scipy
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[data.columns[1:-2]], data['targets'], test_size=0.33, random_state=42)
X_train = torch.tensor(X_train.values).float()
y_train = torch.tensor(y_train.values).float()

In [120]:
X_test = torch.tensor(X_test.values).float()
y_test = torch.tensor(y_test.values).float()

In [121]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
  def __init__(self, values, labels):
    super(MyDataset, self).__init__()
    self.values = values
    self.labels = labels

  def __len__(self):
    return len(self.values)  # number of samples in the dataset

  def __getitem__(self, index):
    return self.values[index], self.labels[index]

In [185]:
from torch.utils.data import DataLoader
dataset = MyDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size)
testset = MyDataset(X_test, y_test)
testloader = DataLoader(testset, batch_size=5, shuffle=True)

### Neural Network related parameters

In [186]:
input_size = 1917   # The image size = 1x10312 = 10312
hidden_size = 50       # The number of nodes at the hidden layer
num_classes = 6       # The number of output classes. In this case, from 1 to 39
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 5       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence
train_test_split = 0.8

In [187]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 10312 (input data) -> 50 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 50 (hidden node) -> 39 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [188]:
model = Net(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [192]:
for epoch in range(num_epochs):
#for epoch in range(0,1):
    for i, (images, labels) in enumerate(dataloader):   # Load a batch of images with its (index, data, class)
        images = Variable(images)         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        #print(images.shape)
        #print(labels)
        #print(i)
        labels = [i-1 for i in labels]
        #print(torch.tensor(labels).reshape(-1, 1).flatten())
        labels = Variable(torch.tensor(labels).reshape(-1, 1).flatten())
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = model(images.float())                             # Forward pass: compute the output class given a image
        #if epoch == num_epochs-1:
        #    print(outputs, labels.long())
        loss = criterion(outputs, labels.long())                 # Compute the loss: difference between the output class and the pre-given label
        
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes

        if (i+1) % 20 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(dataset)//batch_size, loss.item()))

Epoch [1/5], Step [20/66], Loss: 1.6189
Epoch [2/5], Step [20/66], Loss: 1.6227
Epoch [3/5], Step [20/66], Loss: 1.6009
Epoch [4/5], Step [20/66], Loss: 1.6205
Epoch [5/5], Step [20/66], Loss: 1.6015


In [193]:
correct = 0
total = 0
all_predicted = []
all_labels = []
for i, (images, labels) in enumerate(testloader):
    images = Variable(images)
    outputs = model(images.float()) 
    labels = [i-1 for i in labels]
    labels = torch.tensor(labels).reshape(-1, 1).flatten()
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    all_predicted += list(predicted)
    #all_labels += (labels)
    
    
print('Accuracy: %d %%' % (100 * correct / total))

Accuracy: 34 %


### Split data into train/test set